# UNET   
It's a Fully Convolutional Network Model.

It used for Semantic Segmentation.

The goal of semantic image segmentation is to label each pixel of an image with a corresponding class of what is being represented.Because we’re predicting for every pixel in the image, this task is commonly referred to as dense prediction.

# Understanding Convolution, Max Pooling and Transposed Convolution

## 1. Convolution operation

There are two inputs to a convolutional operation.<br/>
i) A 3D volume (input image) of size (nin x nin x channels).<br/>
The output of a convolutional operation is also a 3D volume (also called as output image or feature map) of size<br/>(nout x nout x k).<br/>
ii) A set of ‘k’ filters (also called as kernels or feature extractors) each one of size (f x f x channels),<br/>where f is typically 3 or 5.<br/>
##### Equation for nout = [ (   nin+2p-k  )/s  ]+1 <br/>
nin : no.of input features<br/>
nout : no.of output features<br/>
k : convolution kernel size<br/>
p : convolution padding size<br/>
s : convolution stride size
<br/> for example we are are taking an image of 5 x 5 x 3. The nin=5 and consider the kernal size be 3. Also the padding be 1. The stride size be 2.
then nout=[(5+2*1-3)/2 ]+1 = [(7-3)/2]+1 = [4/2]+1 = 3
<br/> if we are using two filters then then it will be 3 x 3 x 2.

One important term used frequently is called as the **Receptive filed**. This is nothing but the region in the input volume that a particular feature extractor (filter) is looking at. Refer the diagram here http://cs231n.github.io/convolutional-networks/
<br/>
Receptive field (context) is the area of the input image that the filter covers at any given point of time.

## 2) Max pooling operation

In simple words, the function of pooling is to reduce the size of the feature map so that we have fewer parameters in the network.

Basically from every block of the input feature map, we select the maximum pixel value and thus obtain a pooled feature map.
https://www.quora.com/What-is-max-pooling-in-convolutional-neural-networks#
<br/>
Note that the size of the filter and strides are two important hyper-parameters in the max pooling operation.


#### Downsampling
A very important point to note here is that both convolution operation and specially the pooling operation reduce the size of the image. This is called as down sampling. <br/>
Now when we apply the convolution operation again, the filters in the next layer will be able to see larger context, i.e. as we go deeper into the network, the size of the image reduces however the receptive field increases.

Notice that in a typical convolutional network, the height and width of the image gradually reduces (down sampling, because of pooling) which helps the filters in the deeper layers to focus on a larger receptive field (context). However the number of channels/depth (number of filters used) gradually increase which helps to extract more complex features from the image.
Intuitively we can make the following conclusion of the pooling operation. By **down sampling**, the model better understands **“WHAT”** is present in the image, but it loses the information of **“WHERE”** it is present.

## 3)Need for up sampling

The output of semantic segmentation is a complete high resolution image in which all the pixels are classified.

Thus if we use a regular convolutional network with pooling layers and dense layers, we will lose the “WHERE” information and only retain the “WHAT” information which is not what we want. In case of segmentation we need both “WHAT” as well as “WHERE” information.
Hence there is a need to up sample the image, i.e. convert a low resolution image to a high resolution image to recover the “WHERE” information.
In the literature, there are many techniques to up sample an image. Some of them are bi-linear interpolation, cubic interpolation, nearest neighbor interpolation, unpooling, transposed convolution, etc. However in most state of the art networks, transposed convolution is the preferred choice for up sampling an image.

## 4) Transposed Convolution

Transposed convolution (sometimes also called as deconvolution or fractionally strided convolution) is a technique to perform up sampling of an image with learnable parameters.


However, on a high level, transposed convolution is exactly the opposite process of a normal convolution i.e., the input volume is a low resolution image and the output volume is a high resolution image.
In the blog it is nicely explained how a normal convolution can be expressed as a matrix multiplication of input image and filter to produce the output image. By just taking the transpose of the filter matrix, we can reverse the convolution process, hence the name transposed convolution.

The transposed convolution operation forms the same connectivity as the normal convolution but in the backward direction.<br/>

We can use it to conduct up-sampling. Moreover, the weights in the transposed convolution are learnable. So we do not need a predefined interpolation method.

Even though it is called the transposed convolution, it does not mean that we take some existing convolution matrix and use the transposed version. The main point is that the association between the input and the output is handled in the backward fashion compared with a standard convolution matrix (one-to-many rather than many-to-one association).<br/>

As such, the transposed convolution is not a convolution. But we can emulate the transposed convolution using a convolution. We up-sample the input by adding zeros between the values in the input matrix in a way that the direct convolution produces the same effect as the transposed convolution. However, it is less efficient due to the need to add zeros to up-sample the input before the convolution.


# UNET Architecture and Training

The architecture contains two paths. <br/>First path is the contraction path (also called as the encoder) which is used to capture the context in the image. <br/>The encoder is just a traditional stack of convolutional and max pooling layers.<br/> The second path is the symmetric expanding path (also called as the decoder) which is used to enable precise localization using transposed convolutions.<br/> Thus it is an end-to-end fully convolutional network (FCN), i.e. it only contains Convolutional layers and does not contain any Dense layer because of which it can accept image of any size.

Note that in the original paper, the size of the input image is 572x572x3. Hence the size at various locations will differ from that in the original paper but the core components remain the same.